<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#构建图" data-toc-modified-id="构建图-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>构建图</a></span></li><li><span><a href="#GNN" data-toc-modified-id="GNN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GNN</a></span></li><li><span><a href="#QA" data-toc-modified-id="QA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>QA</a></span></li><li><span><a href="#整合" data-toc-modified-id="整合-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>整合</a></span></li></ul></div>

In [1]:
from argparse import Namespace
import json
import torch
from collections import defaultdict

from QA_models import AutoQuestionAnswering
from GNN import GAT_HotpotQA

In [2]:
args = Namespace(
    
    dev_json_path = 'data/HotpotQA/hotpot_dev_distractor_v1.json',
    
    GNN_model_path = 'save_cache_GNN/GNN_HotpotQA_hidden64_heads8_pad300_chunk_first.pt',
    QA_model_path = 'save_cache_permutations/HotpotQA_QA_BiGRU_roberta-base-squad2.pt',
    model_path = 'data/models/roberta-base-squad2',
#     # GNN parameters
    features = 768,
    hidden = 64,
    nclass = 2,
    dropout = 0,
    alpha = 0.3,
    nheads = 8,
    pad_max_num = 300,

    device = 'cuda:0',
    
    pretrained_model_path=0,
    header_mode=0,
    cls_token_id=0,
    
    topN_sents=3,
    max_length=512,
    uncased=False,
    seed=123,
)

In [3]:
with open(args.dev_json_path, 'r', encoding='utf-8') as f1:
    dev_json = json.load(f1)

In [4]:
one_item = dev_json[0]

In [5]:
one_item

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
  ['Scott Derrickson',
   ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinema

# 构建图

In [31]:
from gen_nodes_repr import build_for_one_item

In [32]:
ques_items = build_for_one_item(dev_json[:2], args)

init...


100%|██████████| 2/2 [00:07<00:00,  3.92s/it]


# GNN

In [33]:
classifier = GAT_HotpotQA(features=args.features, hidden=args.hidden, nclass=args.nclass, 
                            dropout=args.dropout, alpha=args.alpha, nheads=args.nheads, 
                            nodes_num=args.pad_max_num)
classifier.to("cuda")
args.GNN_model_path = 'save_cache_GNN/GNN_HotpotQA_hidden64_heads8_pad300_chunk_first.pt'
# checkpoint = torch.load(args.GNN_model_path)
# classifier.load_state_dict(checkpoint['model'])

In [34]:
from datasets import HotpotQA_GNN_Dataset, gen_GNN_batches

In [35]:
dataset = HotpotQA_GNN_Dataset.load_for_eval(ques_items)
dataset.set_parameters(300,0)
dataset.set_split('val')

In [36]:
dataset

HotpotQA GNN Dataset. mode: val. size: 2. max_seq: 300

In [37]:
batch_generator = gen_GNN_batches(dataset, 1, shuffle=False, drop_last=False, device='cuda')
QA_eval_list=[]
for index, batch_dict in enumerate(batch_generator):
    with torch.no_grad():
        logits_sent, logits_para, logits_Qtype = \
                        classifier(batch_dict['feature_matrix'], batch_dict['adj'])

        max_value, max_index = logits_sent.max(dim=-1) # max_index is predict class.
        topN_sent_index_batch = (max_value * batch_dict['sent_mask'].squeeze()).topk(3, dim=-1)[1]
        topN_sent_index_batch = topN_sent_index_batch.squeeze().tolist()
        
    item=ques_items[index]
    info_list = [[item["node_list"][item["node_list"][s_id].parent_id].content_raw, 
                         item["node_list"][s_id].order_in_para,
                        item["node_list"][s_id].content_raw] \
                 for s_id in topN_sent_index_batch]
    
    print(item['id'])
    
    question = item["node_list"][0].content_raw
    sup_sent_id_list = [i[:-1] for i in info_list]
    sup_sent_list = [i[-1] for i in info_list]
    
    print(sup_sent_id_list)
    print(sup_sent_list)
    QA_eval_list.append((question, sup_sent_list))

In [38]:
sup_dict

{'5a8b57f25542995d1e6f1371': [['Doctor Strange (2016 film)', 1],
  ['Woodson, Arkansas', 2],
  ['Adam Collis', 1]],
 '5a8c7595554299585d9e36b6': [['Village accountant', 0],
  ['Charles Craft', 3],
  ['Charles Craft', 4]]}

In [39]:
sup_raw_dict

{'5a8b57f25542995d1e6f1371': [' It is the fourteenth film of the Marvel Cinematic Universe (MCU).',
  ' It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area.',
  ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.'],
 '5a8c7595554299585d9e36b6': ['The Village Accountant (variously known as "Patwari", "Talati", "Patel", "Karnam", "Adhikari", "Shanbogaru","Patnaik" etc.) is an administrative government position found in rural parts of the Indian sub-continent.',
  ' Over the next 25 years, Craft would edit 90 feature-length films.',
  ' In the early 1950s he would switch his focus to the small screen, his first show being "Racket Squad", from 1951–53, for which he was the main editor, editing 93 of the 98 episodes.']}

# QA

In [13]:
from datasets import HotpotQA_QA_Dataset, generate_QA_batches
from transformers import AutoTokenizer
from QA_models import AutoQuestionAnswering

In [14]:
args.header_mode = 'MLP'

tokenizer = AutoTokenizer.from_pretrained(args.model_path, local_files_only=True)
classifier = AutoQuestionAnswering.from_pretrained(model_path=args.model_path,
                                                    header_mode=args.header_mode,
                                                    cls_index=tokenizer.cls_token_id)
classifier = classifier.to(args.device)
# checkpoint = torch.load(args.QA_model_path)
# classifier.load_state_dict(checkpoint['model'])
_ = classifier.eval()

In [42]:
QA_eval_list

[('Were Scott Derrickson and Ed Wood of the same nationality?',
  [' In "Doctor Strange", surgeon Strange learns the mystic arts after a career-ending car accident.',
   ' It stars Ethan Hawke as fictional true-crime writer Ellison Oswalt who discovers a box of home movies in his attic that puts his family in danger.',
   ' With Gunn, he has scored every one of the director\'s films; including "Guardians of the Galaxy", which became one of the highest grossing domestic movies of 2014, and its 2017 sequel.']),
 ('What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?',
  [' As an adult, she was named United States ambassador to Ghana and to Czechoslovakia and also served as Chief of Protocol of the United States.',
   ' It stars Shirley Temple in her final starring role as well as her final film appearance.',
   ' A holder of the post would be the third-highest-ranked Great Officer of State, below the Lord High Steward and the Lord High Ch

In [15]:
datasetQA = HotpotQA_QA_Dataset.load_for_eval(QA_eval_list)
datasetQA.set_parameters(tokenizer = tokenizer, topN_sents = args.topN_sents,
                        max_length=args.max_length, uncased=args.uncased,
                        permutations=False, random_seed=args.seed)
batch_generatorQA = generate_QA_batches(datasetQA, 1, shuffle=False, drop_last=False, device="cuda")

In [16]:
datasetQA

HotpotQA QA Dataset. mode: eval. size: 2. sents num: 3

In [17]:
ans_dict = {}
ans_dict_topN = defaultdict(list)

for index, batch_dict in enumerate(batch_generatorQA):
    print(index)
    with torch.no_grad():
        res = classifier(**batch_dict)
        start_top_log_probs, start_top_index, end_top_log_probs, end_top_index, cls_logits = res[:5]
        start_top_index = start_top_index.squeeze().tolist()
        end_top_index = end_top_index.squeeze().tolist()
        assert len(start_top_index) == len(end_top_index)
        
        input_ids = batch_dict['input_ids'].squeeze().tolist()
        item = ques_items[index]
        for index,(i,j) in enumerate(zip(start_top_index,end_top_index)):
            if index == 0: ans_dict[item['id']] = tokenizer.decode(input_ids[i:j+1])
            ans_dict_topN[item['id']].append(tokenizer.decode(input_ids[i:j+1]))

0
1


In [18]:
ans_dict

{'5a8b57f25542995d1e6f1371': ' Scott Derrickson and Ed Wood of the same nationality</s></s> In Doctor Strange surgeon Strange learns the mystic arts after a career ending car accident   It stars Ethan Hawke as fictional true crime writer Ellison Oswalt who discovers a box of home movies in his attic that puts his family in danger   With Gunn he has scored',
 '5a8c7595554299585d9e36b6': ''}

# 整合

In [40]:
final_res = {}
final_res['answer'] = ans_dict
final_res['sp'] = sup_dict

In [41]:
final_res

{'answer': {'5a8b57f25542995d1e6f1371': ' Scott Derrickson and Ed Wood of the same nationality</s></s> In Doctor Strange surgeon Strange learns the mystic arts after a career ending car accident   It stars Ethan Hawke as fictional true crime writer Ellison Oswalt who discovers a box of home movies in his attic that puts his family in danger   With Gunn he has scored',
  '5a8c7595554299585d9e36b6': ''},
 'sp': {'5a8b57f25542995d1e6f1371': [['Doctor Strange (2016 film)', 1],
   ['Woodson, Arkansas', 2],
   ['Adam Collis', 1]],
  '5a8c7595554299585d9e36b6': [['Village accountant', 0],
   ['Charles Craft', 3],
   ['Charles Craft', 4]]}}